In [11]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

# Turn off GPU
if False:
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from tensorflow import keras
import tensorflow as tf
from utils.model_utils import lastLayerToArgMax
from glob import glob
import numpy as np

gpus = tf.config.experimental.list_physical_devices('GPU')

print(tf.__version__)
print(gpus)

if len(gpus) > 0:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024 * 1))])

2.10.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [12]:
from glob import glob

# last_models = sorted(glob("model-saved/deeplabV3+_mobileNetV2/NEW_4/GRID_SEARCH/*/*/*/*/*/*.ckpt"))
last_models = sorted(glob("model-saved/deeplabV3+_mobileNetV2/NEW_5/LAST_MODEL/*/*/*/*/*/*.ckpt"))

modelPaths = last_models
print(last_models, len(last_models))

['model-saved/deeplabV3+_mobileNetV2/NEW_5/LAST_MODEL\\layer_12\\alpha_0.5\\loss_iou_coef\\l1_0\\l2_0.01\\VERTICAL_FLIP_False.ckpt', 'model-saved/deeplabV3+_mobileNetV2/NEW_5/LAST_MODEL\\layer_12\\alpha_0.5\\loss_iou_coef\\l1_0\\l2_0.01\\VERTICAL_FLIP_True.ckpt'] 2


In [13]:
imageSize = (240, 320)
n_classes = 2

In [17]:
modelPath = sorted(glob("model-saved/deeplabV3+_mobileNetV2/NEW_6/kfold/*/*/*/*/*/*.ckpt"))[0]
model = tf.keras.models.load_model(modelPath, compile=False)

In [14]:
import models
from utils.model_utils import BatchGenerator
from utils.model_utils import toTFlite
from tensorflow import lite as tflite

for i, modelPath in enumerate(modelPaths):
    print(f"{i+1} of {len(last_models)}")
    modelSplit = modelPath.replace("/", "\\").split("\\")

    # TEST = modelSplit[4] == "True"
    # deeplayer = modelSplit[5].replace("layer_", "")
    # alpha = float(modelSplit[6].replace("alpha_", ""))

    TEST = None
    deeplayer = modelSplit[4].replace("layer_", "")
    alpha = float(modelSplit[5].replace("alpha_", ""))

    print(TEST)
    print(deeplayer)
    print(alpha)

    mobileLayers = {
        "shallowLayer": "block_2_project_BN",
        "deepLayer": f"block_{deeplayer}_project_BN"}
    
    # if TEST is not None:
    #     model = models.deeplabV3(
    #         imageSize=imageSize, nClasses=n_classes, alpha=alpha, withArgmax=True, mobileLayers=mobileLayers, TEST=TEST)
    # else:
    #     model = models.deeplabV3Alpha(
    #         imageSize=imageSize, nClasses=n_classes, alpha=alpha, last=0.25, withArgmax=True, mobileLayers=mobileLayers)
    model = tf.keras.models.load_model(modelPath, compile=False)
    model = lastLayerToArgMax(model=model)

    print(modelPath)

    images = glob("utils\\split_3\\TrainVal\\img\\*")
    print(len(images))

    toTFlite(
        model=model,
        savePath=modelPath.replace(".ckpt", ""),
        representativeDatasetGen=BatchGenerator(images),
        supportedOps=[
            tflite.OpsSet.SELECT_TF_OPS,
            tflite.OpsSet.TFLITE_BUILTINS])

1 of 2
None
12
0.5
model-saved/deeplabV3+_mobileNetV2/NEW_5/LAST_MODEL\layer_12\alpha_0.5\loss_iou_coef\l1_0\l2_0.01\VERTICAL_FLIP_False.ckpt
2975


INFO:tensorflow:Assets written to: C:\Users\rafae\AppData\Local\Temp\tmp_294mrsq\assets


INFO:tensorflow:Assets written to: C:\Users\rafae\AppData\Local\Temp\tmp_294mrsq\assets
d:\Anaconda\envs\tf-gpu\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


2 of 2
None
12
0.5


model-saved/deeplabV3+_mobileNetV2/NEW_5/LAST_MODEL\layer_12\alpha_0.5\loss_iou_coef\l1_0\l2_0.01\VERTICAL_FLIP_True.ckpt
2975


INFO:tensorflow:Assets written to: C:\Users\rafae\AppData\Local\Temp\tmp_2jfnebp\assets


INFO:tensorflow:Assets written to: C:\Users\rafae\AppData\Local\Temp\tmp_2jfnebp\assets
d:\Anaconda\envs\tf-gpu\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
